In [29]:
from sibyl.core import Sibyl
from mongoengine import disconnect

In [30]:
database = "mimic"

In [31]:
disconnect()
config = {
    "mongodb": {
        "db": database,
        "host": "localhost",
        "port": 27017,
        "username": None,
        "password": None,
    },
    "log_filename": "test.csv",
    "feature_distribution_location": None,
    "flask": {},
}
explorer = Sibyl(config, docker=False)
app = explorer._init_flask_app("test")
client = app.test_client()

In [32]:
# TEST ENTITY APIs
eid = None

response = client.get("/api/v1/entities/")
assert response.status_code == 200
assert 'entities' in response.json
print("Number of entities:",   len(response.json['entities']))

if eid is None:
    eid = response.json["entities"][0]["eid"]

response = client.get("/api/v1/entities/" + eid + "/")
assert response.status_code == 200
assert response.json["eid"] == eid
assert "features" in response.json
assert "property" in response.json
sample_entity = response.json

Number of entities: 129


In [33]:
# TEST FEATURE APIs

response = client.get("/api/v1/features/")
assert response.status_code == 200
assert "features" in response.json
sample_feature = response.json['features'][0]

response = client.get("/api/v1/features/" + sample_feature["name"] + "/")
assert response.status_code == 200
assert "name" in response.json
print("Sample feature:", response.json)

response = client.get("/api/v1/categories/")
assert response.status_code == 200
assert "categories" in response.json
print("Sample category", response.json["categories"][0])

Sample feature: {'name': 'row_id', 'description': 'row_id', 'type': 'numeric', 'negated_description': None, 'category': 'admissions'}
Sample category {'name': 'admissions', 'color': None, 'abbreviation': 'AD'}


In [34]:
# TEST MODEL APIs
response = client.get("/api/v1/models/")
assert response.status_code == 200
sample_model = response.json["models"][0]
model_id = sample_model["id"]

response = client.get("/api/v1/models/" + model_id + "/")
assert response.status_code == 200
print("Sample model:", response.json)

response = client.get("/api/v1/importance/?model_id=" + model_id)
assert response.status_code == 200
assert "importances" in response.json
print("Sample importance", next(iter(response.json["importances"].items())))

response = client.get("/api/v1/prediction/?model_id=" + model_id + "&eid=" + eid)
assert response.status_code == 200
print("Sample prediction:", response.json)
prediction = response.json["output"]

Sample model: {'id': '6449707db2808295294ae459', 'name': 'placeholder', 'description': 'placeholder', 'performance': 'placeholder'}
Sample importance ('row_id', 0.0)
Sample prediction: {'output': False}


In [35]:
# TEST COMPUTING

response = client.post("/api/v1/contributions/", json={"eid": eid, "model_id": model_id})
assert response.status_code == 200
assert "contributions" in response.json
print("Sample contribution", next(iter(response.json["contributions"].items())))

#response = client.post("/api/v1/prediction_count/", json={"prediction": prediction, "model_id": model_id})
#assert response.status == 200

features_options = list(sample_entity["features"].keys())
changes = [(features_options[0], sample_entity["features"][features_options[0]]),
           (features_options[1], sample_entity["features"][features_options[1]])]
response = client.post("/api/v1/modified_prediction/",json={"eid": eid, "model_id": model_id, "changes": changes})
assert response.status_code == 200
assert "prediction" in response.json
print("Sample modified prediction:", response.json)

response = client.post("/api/v1/single_change_predictions/", json={"eid": eid, "model_id": model_id, "changes": changes})
assert response.status_code == 200
assert "changes" in response.json
print("Sample changes:", response.json["changes"])

Sample contribution ('row_id', 0.0)
Sample modified prediction: {'prediction': False}
Sample changes: [['row_id', False], ['subject_id', False]]
